# Lot Sizing Case Study - Parts 3&4

In [2]:
# Import libraries

from    pulp    import *                # Will be used as the solver 
import  numpy   as np                   
import  pandas  as pd                   # Will be used to generate tables                 


## Part 3

In [3]:
# Definition of the problem

model3 = LpProblem( name    = "lot-sizing", 
                    sense   = LpMinimize            )
                    

In [4]:
# Definition of the sets

T       = range(1,13)       # Set of months:                    T = {1, 2, 3, 4 ... 12}
T_ext   = range(13)         # Extended time horizon             T = 0 included
P       = ("P1", "P2")      # Set of product groups: basic and pro


In [5]:
# Definition of the parameters

D = (   (226,   171), 
        (117,   177),
        (5,     144),
        (11,    192),
        (211,   188),
        (91,    210),
        (147,   185),
        (182,   221),
        (92,    196),
        (36,    208),
        (188,   172),
        (11,    200)     )      # Weekly demands

c       = (2, 3)                # Inventory costs [$ / month]
iInit   = (85, 50)              # Initial inventories

b       = (1, 2)                # Backlog costs
s       = (750, 450)            # Setup costs

R       = 320                   # Weekly resource capacity of the facility
r       = (0.5, 1.3)            # Capacity requirements


In [6]:
# Setting big M

M = max([sum(i) for i in zip(*D)])      # Maximum # of products that is appropriate to order


In [7]:
# Definition of the decision variables

X = {   (i, t): LpVariable( name="X-{model}-{week}".format(     model   = P[i],                                     # Amount of ordered products per week
                                                                week    = t     if t>=10 
                                                                                else "0{}".format(t)),              # Standardizing the time indices with two digits
                            lowBound=0                                                               )
                                                                                for i in range(len(P)) for t in T       }


I = {   (i, t): LpVariable( name="I-{model}-{week}".format(     model   = P[i],                                     # Inventory of item i at the end of week t
                                                                week    = t     if t>=10 
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for i in range(len(P)) for t in T_ext   }

B = {   (i, t): LpVariable( name="B-{model}-{week}".format(     model   = P[i],                                     # Backlog of item i at the end of week t
                                                                week    = t     if t>=10 
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for i in range(len(P)) for t in T_ext   }


Y = {   (i, t): LpVariable( name="Y-{model}-{week}".format(     model   = P[i],                                     # Whether product i is ordered at period t
                                                                week    = t     if t>=10 
                                                                                else "0{}".format(t)),
                            cat="Binary"                                                            )
                                                                                for i in range(len(P)) for t in T       }


In [8]:
# Statement of the objective function: The total cost must be minimized.

model3 += lpSum(
                [   b[i]*B[(i,t)]                   +                           # Backlog costs,       
                    c[i]*(I[(i,t)] + I[(i,t-1)])/2  +                           # Inventory holding costs,
                    s[i]*Y[(i,t)]                                               # and setup costs
                                            for t in T                          # are summed over each month,
                                            for i in range(len(P)   )  ]    )   # and each product group.


In [9]:
# Definition of the constraints

for i in range(len(P)):                                                                         # Setting the initial inventory levels
    model3 += (I[(i, 0)] ==  iInit[i]
                                                                                    )
    
for i in range(len(P)):                                                                         # Setting the final backlog to zero
    model3 += (B[(i, len(T))] ==  0
                                                                                    )

for i in range(len(P)):                                                                         # Inventory balance / demand satisfaction
    for t in T:
        model3 += ( I[(i,t-1)]  - B[(i,t-1)]    + X[(i,t)]  ==
                    I[(i,t)]    - B[(i,t)]      + D[t-1][i]                         )

for i in range(len(P)):
    for t in T:                                                                                 # Binary variable adjustment for fixed costs
        model3 += (  X[(i,t)] <= M*Y[(i,t)]                                          )



In [10]:
model4 = model3

model3

lot-sizing:
MINIMIZE
1*B_P1_01 + 1*B_P1_02 + 1*B_P1_03 + 1*B_P1_04 + 1*B_P1_05 + 1*B_P1_06 + 1*B_P1_07 + 1*B_P1_08 + 1*B_P1_09 + 1*B_P1_10 + 1*B_P1_11 + 1*B_P1_12 + 2*B_P2_01 + 2*B_P2_02 + 2*B_P2_03 + 2*B_P2_04 + 2*B_P2_05 + 2*B_P2_06 + 2*B_P2_07 + 2*B_P2_08 + 2*B_P2_09 + 2*B_P2_10 + 2*B_P2_11 + 2*B_P2_12 + 1.0*I_P1_00 + 2.0*I_P1_01 + 2.0*I_P1_02 + 2.0*I_P1_03 + 2.0*I_P1_04 + 2.0*I_P1_05 + 2.0*I_P1_06 + 2.0*I_P1_07 + 2.0*I_P1_08 + 2.0*I_P1_09 + 2.0*I_P1_10 + 2.0*I_P1_11 + 1.0*I_P1_12 + 1.5*I_P2_00 + 3.0*I_P2_01 + 3.0*I_P2_02 + 3.0*I_P2_03 + 3.0*I_P2_04 + 3.0*I_P2_05 + 3.0*I_P2_06 + 3.0*I_P2_07 + 3.0*I_P2_08 + 3.0*I_P2_09 + 3.0*I_P2_10 + 3.0*I_P2_11 + 1.5*I_P2_12 + 750*Y_P1_01 + 750*Y_P1_02 + 750*Y_P1_03 + 750*Y_P1_04 + 750*Y_P1_05 + 750*Y_P1_06 + 750*Y_P1_07 + 750*Y_P1_08 + 750*Y_P1_09 + 750*Y_P1_10 + 750*Y_P1_11 + 750*Y_P1_12 + 450*Y_P2_01 + 450*Y_P2_02 + 450*Y_P2_03 + 450*Y_P2_04 + 450*Y_P2_05 + 450*Y_P2_06 + 450*Y_P2_07 + 450*Y_P2_08 + 450*Y_P2_09 + 450*Y_P2_10 + 450*Y_P2_11 + 450*Y

In [11]:
len(T_ext)

13

In [12]:
model3.solve()
LpStatus[model3.status]

'Optimal'

In [13]:
print("z* = ", value(model3.objective))

z* =  8425.0


In [14]:
for v in model3.variables():
    print(v.name, " = ", v.varValue)

B_P1_00  =  0.0
B_P1_01  =  141.0
B_P1_02  =  0.0
B_P1_03  =  0.0
B_P1_04  =  11.0
B_P1_05  =  222.0
B_P1_06  =  313.0
B_P1_07  =  0.0
B_P1_08  =  0.0
B_P1_09  =  92.0
B_P1_10  =  128.0
B_P1_11  =  0.0
B_P1_12  =  0.0
B_P2_00  =  0.0
B_P2_01  =  121.0
B_P2_02  =  0.0
B_P2_03  =  144.0
B_P2_04  =  0.0
B_P2_05  =  188.0
B_P2_06  =  0.0
B_P2_07  =  185.0
B_P2_08  =  0.0
B_P2_09  =  196.0
B_P2_10  =  0.0
B_P2_11  =  172.0
B_P2_12  =  0.0
I_P1_00  =  85.0
I_P1_01  =  0.0
I_P1_02  =  5.0
I_P1_03  =  0.0
I_P1_04  =  0.0
I_P1_05  =  0.0
I_P1_06  =  0.0
I_P1_07  =  182.0
I_P1_08  =  0.0
I_P1_09  =  0.0
I_P1_10  =  0.0
I_P1_11  =  11.0
I_P1_12  =  0.0
I_P2_00  =  50.0
I_P2_01  =  0.0
I_P2_02  =  0.0
I_P2_03  =  0.0
I_P2_04  =  0.0
I_P2_05  =  0.0
I_P2_06  =  0.0
I_P2_07  =  0.0
I_P2_08  =  0.0
I_P2_09  =  0.0
I_P2_10  =  0.0
I_P2_11  =  0.0
I_P2_12  =  0.0
X_P1_01  =  0.0
X_P1_02  =  263.0
X_P1_03  =  0.0
X_P1_04  =  0.0
X_P1_05  =  0.0
X_P1_06  =  0.0
X_P1_07  =  642.0
X_P1_08  =  0.0
X_P1_09  

## Part 4

In [15]:
# Resource capacity for the forth part        

for t in T:                             
    model4 += (  lpSum(
                        [ X[(i,t)]*r[i] for i in range(len(P))  ]  
                                                                  ) <= R)           


In [16]:
model4

lot-sizing:
MINIMIZE
1*B_P1_01 + 1*B_P1_02 + 1*B_P1_03 + 1*B_P1_04 + 1*B_P1_05 + 1*B_P1_06 + 1*B_P1_07 + 1*B_P1_08 + 1*B_P1_09 + 1*B_P1_10 + 1*B_P1_11 + 1*B_P1_12 + 2*B_P2_01 + 2*B_P2_02 + 2*B_P2_03 + 2*B_P2_04 + 2*B_P2_05 + 2*B_P2_06 + 2*B_P2_07 + 2*B_P2_08 + 2*B_P2_09 + 2*B_P2_10 + 2*B_P2_11 + 2*B_P2_12 + 1.0*I_P1_00 + 2.0*I_P1_01 + 2.0*I_P1_02 + 2.0*I_P1_03 + 2.0*I_P1_04 + 2.0*I_P1_05 + 2.0*I_P1_06 + 2.0*I_P1_07 + 2.0*I_P1_08 + 2.0*I_P1_09 + 2.0*I_P1_10 + 2.0*I_P1_11 + 1.0*I_P1_12 + 1.5*I_P2_00 + 3.0*I_P2_01 + 3.0*I_P2_02 + 3.0*I_P2_03 + 3.0*I_P2_04 + 3.0*I_P2_05 + 3.0*I_P2_06 + 3.0*I_P2_07 + 3.0*I_P2_08 + 3.0*I_P2_09 + 3.0*I_P2_10 + 3.0*I_P2_11 + 1.5*I_P2_12 + 750*Y_P1_01 + 750*Y_P1_02 + 750*Y_P1_03 + 750*Y_P1_04 + 750*Y_P1_05 + 750*Y_P1_06 + 750*Y_P1_07 + 750*Y_P1_08 + 750*Y_P1_09 + 750*Y_P1_10 + 750*Y_P1_11 + 750*Y_P1_12 + 450*Y_P2_01 + 450*Y_P2_02 + 450*Y_P2_03 + 450*Y_P2_04 + 450*Y_P2_05 + 450*Y_P2_06 + 450*Y_P2_07 + 450*Y_P2_08 + 450*Y_P2_09 + 450*Y_P2_10 + 450*Y_P2_11 + 450*Y

In [17]:
model4.solve()
LpStatus[model4.status]

'Optimal'

In [18]:
print("z* = ", value(model4.objective))

z* =  9553.153853500002


In [19]:
for v in model4.variables():
    print(v.name, " = ", v.varValue)

B_P1_00  =  0.0
B_P1_01  =  141.0
B_P1_02  =  0.0
B_P1_03  =  0.0
B_P1_04  =  9.0
B_P1_05  =  220.0
B_P1_06  =  311.0
B_P1_07  =  0.0
B_P1_08  =  0.0
B_P1_09  =  92.0
B_P1_10  =  128.0
B_P1_11  =  0.0
B_P1_12  =  0.0
B_P2_00  =  0.0
B_P2_01  =  0.0
B_P2_02  =  102.15385
B_P2_03  =  0.0
B_P2_04  =  0.0
B_P2_05  =  0.0
B_P2_06  =  0.0
B_P2_07  =  90.692308
B_P2_08  =  65.538462
B_P2_09  =  15.384615
B_P2_10  =  0.0
B_P2_11  =  46.153846
B_P2_12  =  0.0
I_P1_00  =  85.0
I_P1_01  =  0.0
I_P1_02  =  7.0
I_P1_03  =  2.0
I_P1_04  =  0.0
I_P1_05  =  0.0
I_P1_06  =  0.0
I_P1_07  =  182.0
I_P1_08  =  0.0
I_P1_09  =  0.0
I_P1_10  =  0.0
I_P1_11  =  11.0
I_P1_12  =  0.0
I_P2_00  =  50.0
I_P2_01  =  74.846154
I_P2_02  =  0.0
I_P2_03  =  0.0
I_P2_04  =  0.0
I_P2_05  =  58.153846
I_P2_06  =  94.307692
I_P2_07  =  0.0
I_P2_08  =  0.0
I_P2_09  =  0.0
I_P2_10  =  5.4615385
I_P2_11  =  0.0
I_P2_12  =  0.0
X_P1_01  =  0.0
X_P1_02  =  265.0
X_P1_03  =  0.0
X_P1_04  =  0.0
X_P1_05  =  0.0
X_P1_06  =  0.0
X_